In [1]:
!pip install transformers==4.46.3
!pip install accelerate==0.26.0

In [2]:
!pip install datasets tiktoken sentencepiece protobuf

In [3]:
import numpy as np
import pandas as pd
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [30]:
df_books = pd.read_csv("code/data/books.csv")
df_users = pd.read_csv("code/data/users.csv")

#print(df_books.head())

Images 대신 Images_medium으로 변경

In [5]:
df_books['img_path'] = (
    df_books['img_path']
    .str.replace(r'^images/', 'images_medium/', regex=True)
    .str.replace(r'THUMBZZZ', 'MZZZZZZZ', regex=False)
)

In [32]:
df_books.head()

,isbn,book_title,book_author,year_of_publication,publisher,img_url,language,category,summary,img_path
0,0002005018,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,en,['Actresses'],"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg
1,0060973129,Decision in Normandy,Carlo D'Este,1991.0,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,en,['1940-1949'],"Here, for the first time in paperback, is an o...",images/0060973129.01.THUMBZZZ.jpg
2,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,en,['Medical'],"Describes the great flu epidemic of 1918, an o...",images/0374157065.01.THUMBZZZ.jpg
3,0399135782,The Kitchen God's Wife,Amy Tan,1991.0,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,en,['Fiction'],A Chinese immigrant who is convinced she is dy...,images/0399135782.01.THUMBZZZ.jpg
4,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000.0,Berkley Publishing Group,http://images.amazon.com/images/P/0425176428.0...,en,['History'],"Essays by respected military historians, inclu...",images/0425176428.01.THUMBZZZ.jpg


In [7]:
#df_books.to_csv("./code/data/books_medium.csv", index=False)

In [8]:
print(df_books.isnull().sum())
print(len(df_books))

isbn                       0
book_title                 0
book_author                1
year_of_publication        0
publisher                  0
img_url                    0
language               67227
category               68851
summary                67227
img_path                   0
dtype: int64
149570


In [9]:
print(df_books['category'].value_counts()) # language, category, summary

category
['Fiction']                        32956
['Juvenile Fiction']                5804
['Biography & Autobiography']       3320
['History']                         1925
['Religion']                        1818
                                   ...  
['Disguise']                           1
['Country lawyers']                    1
['Humorous stories, Brazilian']        1
['Coasts']                             1
['Vocational guidance']                1
Name: count, Length: 4292, dtype: int64


In [10]:
print(df_users.head())

   user_id                            location   age
0        8            timmins, ontario, canada   NaN
1    11400             ottawa, ontario, canada  49.0
2    11676                       n/a, n/a, n/a   NaN
3    67544            toronto, ontario, canada  30.0
4    85526  victoria, british columbia, canada  36.0


In [11]:
print(df_users.isnull().sum())
print(len(df_users))

user_id         0
location        0
age         27833
dtype: int64
68092


In [12]:
print(df_users['age'].value_counts().to_string())

age
29.0    1290
26.0    1250
25.0    1243
34.0    1215
27.0    1207
33.0    1200
32.0    1195
28.0    1192
30.0    1182
35.0    1167
24.0    1142
31.0    1142
23.0    1112
36.0    1023
37.0     964
22.0     908
38.0     890
21.0     882
39.0     846
40.0     807
41.0     775
19.0     761
18.0     760
43.0     753
20.0     743
44.0     742
42.0     683
17.0     680
45.0     660
46.0     634
47.0     630
50.0     618
49.0     611
51.0     586
16.0     572
48.0     563
54.0     554
53.0     546
52.0     544
57.0     517
15.0     471
56.0     463
55.0     453
14.0     421
58.0     346
59.0     333
60.0     327
61.0     300
62.0     243
63.0     228
13.0     194
66.0     193
64.0     184
65.0     179
67.0     132
68.0     123
70.0     113
69.0     109
71.0      76
72.0      64
73.0      62
74.0      58
12.0      48
75.0      44
11.0      43
76.0      40
9.0       26
10.0      26
77.0      26
8.0       24
80.0      17
78.0      15
79.0      13
81.0      12
5.0       10
7.0       10
83.0    

Category 결측치 학습

In [33]:
df_books['category'] = df_books['category'].apply(lambda x: x[1:-1].split(', ')[0] if not pd.isna(x) else np.nan)
df_books['category_is_missing'] = df_books['category'].isnull().astype(int)
df_books['category'] = df_books['category'].fillna('Unknown')

In [34]:
df_books.head(10)

,isbn,book_title,book_author,year_of_publication,publisher,img_url,language,category,summary,img_path,category_is_missing
0,0002005018,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,en,'Actresses',"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg,0
1,0060973129,Decision in Normandy,Carlo D'Este,1991.0,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,en,'1940-1949',"Here, for the first time in paperback, is an o...",images/0060973129.01.THUMBZZZ.jpg,0
2,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,en,'Medical',"Describes the great flu epidemic of 1918, an o...",images/0374157065.01.THUMBZZZ.jpg,0
3,0399135782,The Kitchen God's Wife,Amy Tan,1991.0,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,en,'Fiction',A Chinese immigrant who is convinced she is dy...,images/0399135782.01.THUMBZZZ.jpg,0
4,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000.0,Berkley Publishing Group,http://images.amazon.com/images/P/0425176428.0...,en,'History',"Essays by respected military historians, inclu...",images/0425176428.01.THUMBZZZ.jpg,0
5,0671870432,PLEADING GUILTY,Scott Turow,1993.0,Audioworks,http://images.amazon.com/images/P/0671870432.0...,NaN,Unknown,NaN,images/0671870432.01.THUMBZZZ.jpg,1
6,074322678X,Where You'll Find Me: And Other Stories,Ann Beattie,2002.0,Scribner,http://images.amazon.com/images/P/074322678X.0...,en,'Fiction',"Now back in print, Ann Beattie&#39;s finest sh...",images/074322678X.01.THUMBZZZ.jpg,0
7,0771074670,Nights Below Station Street,David Adams Richards,1988.0,Emblem Editions,http://images.amazon.com/images/P/0771074670.0...,en,'Fiction',Another story based in the fictional rural tow...,images/0771074670.01.THUMBZZZ.jpg,0
8,0887841740,The Middle Stories,Sheila Heti,2004.0,House of Anansi Press,http://images.amazon.com/images/P/0887841740.0...,NaN,Unknown,NaN,images/0887841740.01.THUMBZZZ.jpg,1
9,1552041778,Jane Doe,R. J. Kaiser,1999.0,Mira Books,http://images.amazon.com/images/P/1552041778.0...,NaN,Unknown,NaN,images/1552041778.01.THUMBZZZ.jpg,1


In [35]:
df_books['category'].value_counts()

category
Unknown                                           68851
'Fiction'                                         32956
'Juvenile Fiction'                                 5804
'Biography & Autobiography'                        3320
'History'                                          1925
                                                  ...  
'Arabian nights'                                      1
'Cleaning'                                            1
'Bread dough craft'                                   1
'Publicity'                                           1
"Chefs d'orchestre - États-Unis - Biographies"        1
Name: count, Length: 4121, dtype: int64

In [36]:
from transformers import DataCollatorWithPadding

# ============================================
# 1. 데이터 준비
# ============================================
# 학습 데이터: Category가 있는 데이터
train_subset = df_books[df_books['category'] != 'Unknown'].copy()
# 예측 데이터: Category가 없는 데이터
pred_subset = df_books[df_books['category'] == 'Unknown'].copy()

# 입력 테스트 생성 (Title + Summary)
train_subset['text'] = (train_subset['book_title'] + " " + train_subset['summary'].fillna('')).astype(str)
pred_subset['text'] = (pred_subset['book_title'] + " " + pred_subset['summary'].fillna('')).astype(str)

# ============================================
# 2. 레이블 인코딩 (문자열 -> 숫자)
# ============================================
le = LabelEncoder()
train_subset['label'] = le.fit_transform(train_subset['category'])
num_labels = len(le.classes_)

# Hugging Face Dataset 변환
hf_train = Dataset.from_pandas(train_subset[['text', 'label']])
hf_pred = Dataset.from_pandas(pred_subset[['text']])

# ============================================
# 3. 토크나이저 및 모델 로드 (DeBERTa-v3-base)
# ============================================
model_name = "microsoft/deberta-v3-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

# 토크나이징 함수
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=256)

tokenized_train = hf_train.map(tokenize_function, batched=True)
tokenized_pred = hf_pred.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# ============================================
# 4. 학습 설정
# ============================================
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    dataloader_num_workers=4,
    dataloader_pin_memory=True,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="no",
    save_strategy="no",
    load_best_model_at_end=False,
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    tokenizer=tokenizer,
    data_collator=data_collator
)

/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/80719 [00:00<?, ? examples/s]

Map:   0%|          | 0/68851 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_445004/3367462138.py:61: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [37]:
# ============================================
# 5. 학습 시작
# ============================================
print("학습 시작...")
trainer.train()

학습 시작...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Step,Training Loss
500,3.791200
1000,2.770600
1500,2.555700
2000,2.359200
2500,2.354300


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

TrainOutput(global_step=2524, training_loss=2.7624284363397518, metrics={'train_runtime': 415.1383, 'train_samples_per_second': 388.878, 'train_steps_per_second': 6.08, 'total_flos': 7280355747175536.0, 'train_loss': 2.7624284363397518, 'epoch': 2.0})

In [38]:
print(tokenized_train)
print(tokenized_pred)
print(tokenized_train[0])

Dataset({
    features: ['text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 80719
})
Dataset({
    features: ['text', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 68851
})
{'text': 'Clara Callan In a small town in Canada, Clara Callan reluctantly takes leave of her\nsister, Nora, who is bound for New York.', 'label': 132, '__index_level_0__': 0, 'input_ids': [1, 15382, 90298, 344, 266, 536, 1240, 267, 1558, 261, 15382, 90298, 28035, 1046, 1021, 265, 342, 2649, 261, 25113, 261, 328, 269, 4935, 270, 485, 920, 260, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [39]:
# ============================================
# 6. 예측 데이터 정리
# ============================================
print("예측 데이터 정리 중...")

# input_ids, attention_mask 제외 column 삭제
keep_cols = ["input_ids", "attention_mask", "token_type_ids"]
cols_to_remove = [col for col in tokenized_pred.column_names if col not in keep_cols]

tokenized_pred_clean = tokenized_pred.remove_columns(cols_to_remove)

# ============================================
# 7. 결측치 예측
# ============================================
print("예측 중...")
predictions = trainer.predict(tokenized_pred_clean)
preds = np.argmax(predictions.predictions, axis=-1)

# ============================================
# 8. 결과 채워넣기
# ============================================
pred_decoded = le.inverse_transform(preds)
df_books.loc[df_books['category'] == 'Unknown', 'category'] = pred_decoded

print("완료! 샘플 확인: ")
print(df_books[df_books['isbn'].isin(pred_subset['isbn'])][['book_title', 'category']].head(10))

예측 데이터 정리 중...
예측 중...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

완료! 샘플 확인: 
                                           book_title            category
5                                     PLEADING GUILTY           'Fiction'
8                                  The Middle Stories           'Fiction'
9                                            Jane Doe           'Fiction'
10  A Second Chicken Soup for the Woman's Soul (Ch...  'Health & Fitness'
11       The Witchfinder (Amos Walker Mystery Series)           'Fiction'
25                                    Winter Solstice           'Fiction'
27                        Bant/Spec.Last of the Breed           'Fiction'
30                              Piercing the Darkness           'Fiction'
31                                            Prophet           'Fiction'
42                                           Icebound           'Fiction'


In [40]:
# 1. 전체 카테고리 빈도 확인
category_counts = df_books['category'].value_counts()

# 2. 상위 500개 카테고리만 선정
top_500_categories = category_counts.nlargest(500).index.tolist()

# 3. 상위 500개에 없으면 'Others'로 통합
df_books['category'] = df_books['category'].apply(lambda x: x if x in top_500_categories else 'Others')

print(df_books['category'].nunique())

501


In [41]:
df_books.head()


,isbn,book_title,book_author,year_of_publication,publisher,img_url,language,category,summary,img_path,category_is_missing
0,0002005018,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,en,'Actresses',"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg,0
1,0060973129,Decision in Normandy,Carlo D'Este,1991.0,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,en,Others,"Here, for the first time in paperback, is an o...",images/0060973129.01.THUMBZZZ.jpg,0
2,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,en,'Medical',"Describes the great flu epidemic of 1918, an o...",images/0374157065.01.THUMBZZZ.jpg,0
3,0399135782,The Kitchen God's Wife,Amy Tan,1991.0,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,en,'Fiction',A Chinese immigrant who is convinced she is dy...,images/0399135782.01.THUMBZZZ.jpg,0
4,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000.0,Berkley Publishing Group,http://images.amazon.com/images/P/0425176428.0...,en,'History',"Essays by respected military historians, inclu...",images/0425176428.01.THUMBZZZ.jpg,0


In [42]:
df_books.to_csv("./code/data/books_v4.csv", index=False)

In [26]:
df_books_final = pd.read_csv("code/data/books_medium.csv")
df_books_final['category_is_missing'] = df_books_final['category'].isnull().astype(int)

In [29]:
df_books_final.head(10)

,isbn,book_title,book_author,year_of_publication,publisher,img_url,language,category,summary,img_path,category_is_missing
0,0002005018,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,en,['Actresses'],"In a small town in Canada, Clara Callan reluct...",images_medium/0002005018.01.MZZZZZZZ.jpg,0
1,0060973129,Decision in Normandy,Carlo D'Este,1991.0,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,en,['1940-1949'],"Here, for the first time in paperback, is an o...",images_medium/0060973129.01.MZZZZZZZ.jpg,0
2,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,en,['Medical'],"Describes the great flu epidemic of 1918, an o...",images_medium/0374157065.01.MZZZZZZZ.jpg,0
3,0399135782,The Kitchen God's Wife,Amy Tan,1991.0,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,en,['Fiction'],A Chinese immigrant who is convinced she is dy...,images_medium/0399135782.01.MZZZZZZZ.jpg,0
4,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000.0,Berkley Publishing Group,http://images.amazon.com/images/P/0425176428.0...,en,['History'],"Essays by respected military historians, inclu...",images_medium/0425176428.01.MZZZZZZZ.jpg,0
5,0671870432,PLEADING GUILTY,Scott Turow,1993.0,Audioworks,http://images.amazon.com/images/P/0671870432.0...,NaN,NaN,NaN,images_medium/0671870432.01.MZZZZZZZ.jpg,1
6,074322678X,Where You'll Find Me: And Other Stories,Ann Beattie,2002.0,Scribner,http://images.amazon.com/images/P/074322678X.0...,en,['Fiction'],"Now back in print, Ann Beattie&#39;s finest sh...",images_medium/074322678X.01.MZZZZZZZ.jpg,0
7,0771074670,Nights Below Station Street,David Adams Richards,1988.0,Emblem Editions,http://images.amazon.com/images/P/0771074670.0...,en,['Fiction'],Another story based in the fictional rural tow...,images_medium/0771074670.01.MZZZZZZZ.jpg,0
8,0887841740,The Middle Stories,Sheila Heti,2004.0,House of Anansi Press,http://images.amazon.com/images/P/0887841740.0...,NaN,NaN,NaN,images_medium/0887841740.01.MZZZZZZZ.jpg,1
9,1552041778,Jane Doe,R. J. Kaiser,1999.0,Mira Books,http://images.amazon.com/images/P/1552041778.0...,NaN,NaN,NaN,images_medium/1552041778.01.MZZZZZZZ.jpg,1


In [27]:
df_books_final.to_csv("./code/data/books_medium_final.csv", index=False)